In [1]:
from TrajGen import (
    TrajectoryGenerator,
    DataPoint,
    TrajectoryGeneratorCollection,
    WayPoint,
    color_hex,
    rand_24_bit,
    plot_trajectory,
    plot_datapoint,
    setup_map,
)
import os
import gpxpy
import folium
import random
from typing import Tuple, List
from folium import plugins
import matplotlib.pyplot as plt
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import uuid
import utm 
import gpxpy
import json
from shapely.geometry import Point, Polygon, LineString, MultiPolygon
from shapely import ops

center = [55.39594, 10.38831]

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

In [3]:
kmh_to_ms = (1000 / (60 * 60))
# Set trajectory charateristics
mean_speed = 4.0 * kmh_to_ms  # Average human walking speed in m/s
std_speed = 0.5 * kmh_to_ms   # Deviation in walking speed in m/s
mean_time_delta = 5.
std_time_delta = 20.

def ncrands(n: int, min_: int, max_: int):
    rs = []; r0 = np.random.randint(min_, max_)
    for _ in range(n):
        r1 = np.random.randint(min_, max_ - 1) 
        r2 = (r0 + r1 + 1) % max_
        r0 = r2
        rs.append(r2)
    return rs

def generate_pattern(profile):
    is_anom = profile[0]
    n_patterns = profile[1]
    pattern = profile[2]
    start_pair = WayPoint(
        latitude = pattern[0][0],
        longitude = pattern[0][1],
        duration = 60, # In seconds
        std = 0, # In meters
    )
    end_pair = WayPoint(
        latitude = pattern[-1][0],
        longitude = pattern[-1][1],
        duration = 60, # In seconds
        std = 0, # In meters
    )
    middle_pairs = []
    for i in range(1, len(pattern) - 1):
        middle_pairs.append(
            WayPoint(
                latitude = pattern[i][0],
                longitude = pattern[i][1],
                duration = 0, # Seconds
                std = 0, # In meters
            )
        )
    list_waypoints = [(is_anom, n_patterns, 
            [start_pair] + middle_pairs + [end_pair]
        )
    ]
    return list_waypoints

In [4]:
locations = {    
    "home": [55.367946094149396, 10.405759610262596],
    "fakta": [55.364467731441266, 10.392707986720698],
    "lidl": [55.37274058513472, 10.40862169924273],
    "land": [55.36595581773891, 10.395365460132705],
    "hcl": [55.37298467679492, 10.39972562981267],
    "uni": [55.368209519523774, 10.42749124096433],
    "netto": [55.37495502665289, 10.427400972127419],
}

profiles = {
    #### to land
    "home-land-norm":  [False, 11, [locations["home"], locations["land"]]],
    "home-land-devi1": [True, 1,
        [
            locations["home"],
            [55.367116410064604, 10.394117856530032],
            locations["land"],
        ]
    ],
    # 6 home --> land
    #### to fakta
    "land-fakta-norm":  [False, 12, [locations["land"], locations["fakta"]]],
    "home-fakta-norm": [False, 12, [locations["home"], locations["fakta"]]],
    "home-fakta-devi1": [True, 1,
        [
            locations["home"],
            [55.3688962638592, 10.3985793181517],
            [55.3653719858155, 10.398257453074862],
            locations["fakta"],
        ]
    ],
    "home-fakta-devi2": [True, 1,
        [
            locations["home"],
            [55.3687011563267, 10.401390273156068],
            [55.36571345209057, 10.401175696438177],
            locations["fakta"],
        ]
    ],
    "land-fakta-devi1":  [True, 1, 
        [
            locations["land"],
            [55.36663486286559, 10.385983240440687],
            [55.3647143257673, 10.386424969047207],
            [55.36403646689263, 10.389494982862532],
            locations["fakta"],
        ]
    ],
    "land-fakta-devi2":  [True, 1, 
        [
            locations["land"],
            [55.36650934041169, 10.390599304378837],
            [55.369057368236, 10.389406637141228],
            [55.369747692355794, 10.393757663915466],
            locations["fakta"],
        ]
    ],
    # 10 fakta --> home
    #### from fakta
    "fakta-home-norm": [False, 23,
        [
            locations["fakta"],
            [55.36444916499685, 10.405136107684182],
            locations["home"],
        ]
    ],
    ##
    "fakta-home-devi1": [True, 1,
        [
            locations["fakta"],
            [55.36243178318969, 10.40737547068215],
            locations["home"],
        ]
    ],
    "fakta-home-devi2": [True, 1,
        [
            locations["fakta"],
            [55.36278400458655, 10.404573436222652],
            [55.36478675391273, 10.406346657883642],
            [55.36535525783513, 10.407642473712826],
            locations["home"],
        ]
    ],
    "fakta-home-devi3": [True, 1,
        [
            locations["fakta"],
            [55.3629572473624, 10.403128344570984],
            [55.36487197631148, 10.403793532396445],
            [55.36544515740819, 10.405531603811362],
            locations["home"],
        ]
    ],
    "fakta-home-devi4": [True, 1,
        [
            locations["fakta"],
            [55.36267250064205, 10.397291211737615],
            [55.35988286163377, 10.39727316805758],
            [55.361964853475, 10.403281713508438],
            [55.36625156567692, 10.405861959753098],
            locations["home"],
        ]
    ],
    "fakta-home-devi5": [True, 1,
        [
            locations["fakta"],
            [55.36231061367746, 10.393748256657942],
            [55.361967249170185, 10.403335105790346],
            [55.36614713623522, 10.405777782418603],
            locations["home"],
        ],
    ],
    ####
    "home-uni": [False, 12 + 12,
        [
            locations["home"],
            locations["uni"],
        ]
    ],
    "uni-home": [False, 12,
        [
            locations["uni"],
            locations["home"],
        ]
    ],
    ####
    "uni-uni": [False, 15,
        [
            locations["uni"],
            [55.36607048261808, 10.41911992918521],
            locations["uni"],
        ]
    ],
    "uni-uni-devi1": [True, 1,
        [
            locations["uni"],
            [55.36706461188998, 10.428224174356421],
            locations["uni"],
        ]
    ],
    ####
    "uni-netto": [False, 12,
        [
            locations["uni"],
            locations["netto"],
        ]
    ],
    "netto-lidl": [False, 8,   
        [
            locations["netto"],
            [55.37430882819736, 10.419526006255474],
            locations["lidl"],
        ]
    ], 
    ###
    "netto-lidl-dev1": [True, 1,
        [
            locations["netto"],
            [55.37430882819736, 10.419526006255474],
            [55.375186680133844, 10.419204141302785],
            locations["lidl"],
        ]
    ],
    "netto-lidl-dev2": [True, 1,
        [
            locations["netto"],
            [55.37768651143561, 10.417936038444099],
            [55.3771347840795, 10.40877162746623],
            locations["lidl"],
        ]
    ],
    "netto-lidl-dev3": [True, 1,
        [
            locations["netto"],
            [55.37566385963514, 10.427472022680385],
            [55.37415489357414, 10.417473458120758],
            locations["lidl"],
        ]
    ],
    "netto-lidl-dev4": [True, 1,
        [
            locations["netto"],
            [55.37397937146365, 10.414463078620756],
            [55.37631649139682, 10.41398410975295],
            [55.37622224973701, 10.40919955677602],            
            locations["lidl"],
        ]
    ],
    # 12 netto --> lidl
    ####
    "lidl-home-norm":  [False, 10 + 24 + 6, [locations["lidl"], locations["home"]]],
    "home-lidl-norm":  [False, 17, [locations["home"], locations["lidl"]]],
    "home-lidl-devi1": [True, 1,
        [
            locations["home"],
            [55.368654525477595, 10.40214269932413],
            [55.3708337247534, 10.402759934790717],
            locations["lidl"],
        ]
    ],
    "home-lidl-devi2": [True, 1,
        [
            locations["home"],
            [55.370331240589394, 10.406725459642242],
            [55.37128103764875, 10.406597871562429],
            locations["lidl"],
        ]
    ],
    "home-lidl-devi3": [True, 1,
        [
            locations["home"],
            [55.3689231075168, 10.398002999198704],
            [55.37116098319547, 10.40152045581184],
            locations["lidl"],
        ]
    ],
    "home-lidl-devi4": [True, 1,
        [
            locations["home"],
            [55.369053472678395, 10.393548828596202],
            [55.37191053461741, 10.395020807722354],
            [55.37123702529128, 10.401443989363726],
            locations["lidl"],
        ]
    ],
    "home-lidl-dev5": [True, 1,
        [
            locations["home"],
            [55.36920362839035, 10.393375034104885],
            [55.37764100710188, 10.396407510431787],
            locations["lidl"]
        ]
    ],
    #####    
    "hcl-lidl-norm": [False, 5,
        [
            locations["hcl"],
            [55.37408034921968, 10.403112588773388],
            locations["lidl"],
        ]
    ],
    "hcl-lidl-dev6": [True, 1,
        [
            locations["hcl"],
            [55.378163310200705, 10.403420356235491],
            locations["lidl"]
        ]
    ],
    ####
    "hcl-home-devi1": [True, 1,
        [
            locations["hcl"],
            [55.37124927486343, 10.401271583186231],
            [55.37015850074101, 10.406448780293063],
            locations["home"],
        ]
    ],
    ####
    "home-hcl-norm": [False, 8,
        [
            locations["home"],
            [55.36922148996438, 10.393627298339139],
            locations["hcl"],
        ]
    ],
    "home-hcl-dev1": [True, 1,
        [
            locations["home"],
            [55.369061123671486, 10.393362959756557],
            [55.3739168769067, 10.387490391658314],
            locations["hcl"],
        ],
    ],
    ###
}

In [5]:
path_counter_anom = 0
path_counter_norm = 0
for profile in profiles:
    if profiles[profile][0] == True:
        path_counter_anom += profiles[profile][1]
    else:
        path_counter_norm += profiles[profile][1]
print("N paths: ", path_counter_anom + path_counter_norm)
print("Norm   : ", path_counter_norm)
print("Anom   : ", path_counter_anom)
print("Ratio  : ", path_counter_anom / (path_counter_anom + path_counter_norm))

N paths:  222
Norm   :  199
Anom   :  23
Ratio  :  0.1036036036036036


Add these paragraphs to article:

The large case consists of a total of 222 different geospatial trajectories. 23 are generated as being anomalous and 199 are generated as being normal. In other terms, around 90% of the trajectories are normal and 10% anomalous.

The dection delay is computed as difference between the first point in an anomalous trajectory that can be said to deviate from a group of normal trajectories and the point in an anomalous trajectory at which it has been detected by an apporach as being anomalous. 

In [7]:
patterns_waypoint_list = []

for profile in profiles:
    patterns_waypoint_list.extend(
        generate_pattern(
            profile = profiles[profile],
        )
    )
patterns_waypoint_list

map_ = setup_map(center)

# map_ = plot_datapoint([start_latitude, start_longitude], color = "red", map_ = map_)
# map_ = plot_datapoint([middle_latitude, middle_longitude], color = "green", map_ = map_)
# map_ = plot_datapoint([end_latitude, end_longitude], color = "green", map_ = map_)

trajectories1 = []
for anomaly, n, waypoints in patterns_waypoint_list:
    for _ in range(n):
        random.seed(time.time())
        color = "#" + str(color_hex(num = rand_24_bit()))
        print(waypoints)
        trajectory_generator = TrajectoryGenerator(
            waypoints = waypoints,
            mean_time_delta = mean_time_delta,
            std_time_delta = std_time_delta,
            mean_speed = mean_speed,
            std_speed = std_speed,
            # https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0219890
            # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6638960/
            # np.mean(np.abs(np.random.normal(0, 9, size=1000))) approx 7
            std_datapoint = 8.75,
        )
        map_ = plot_trajectory(
            trajectory_generator.to_latlon(),
            color = color,
            map_ = map_,
        )
        trajectories1.append((anomaly, trajectory_generator))
map_

[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]
[<TrajGen.WayPoint object at 0x7f3cd120c610>, <TrajGen.WayPoint object at 0x7f3cd120c640>]

[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]
[<TrajGen.WayPoint object at 0x7f3cd118dd90>, <TrajGen.WayPoint object at 0x7f3cd118df10>]

[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]
[<TrajGen.WayPoint object at 0x7f3ccceb25b0>, <TrajGen.WayPoint object at 0x7f3ccceb2e50>]

In [8]:
def generate_trajectories(
    waypoints: List,
    start_datetime: datetime,
    gap: timedelta = timedelta(days = 1.),
    *args, **kwargs,
    ) -> TrajectoryGeneratorCollection: 
    trajectories = []
    for anomaly, n, waypoints in waypoints:
        for _ in range(n):
            random.seed(time.time())
            color = "#" + str(color_hex(num = rand_24_bit()))
            trajectory_generator = TrajectoryGenerator(waypoints = waypoints, *args, **kwargs)
            trajectories.append((anomaly, trajectory_generator))
    return TrajectoryGeneratorCollection(
        trajectory_generators = trajectories,
        start_datetime = start_datetime,
        gap = gap,
    ) 

In [9]:
# def generate_batches_deviating(n_batches, seeds):
#     batches = []
#     for seed in seeds:
#         waypoints = generate_deviating_patterns(
#             seed = seed,
#             n_normal = 10,
#             n_deviating = 5, 
#         ) 
#         kwargs = {
#             "mean_time_delta": mean_time_delta,
#             "std_time_delta": std_time_delta,
#             "mean_speed": mean_speed,
#             "std_speed": std_speed,
#             "std_datapoint": 9, # 9
#         }
#         tgc = generate_trajectories(
#             waypoints = waypoints, 
#             start_datetime = datetime.now(),
#             **kwargs,
#         )
#         main_df = tgc.to_dataframe(anom_start = True)
#         batches.append(main_df.to_json(orient = "split"))
#     return batches




# tgc = TrajectoryGeneratorCollection(
#         trajectory_generators = trajectories1,
#         start_datetime = datetime.now(),
#         gap = timedelta(days = 1.),
#     ) 
# main_df = tgc.to_dataframe(anom_start = True)




# def read_batches(data_in):
# #     data_in = None; #df = 
# #     with open(json_file, "r") as f:
# #         data_in = json.loads(f.read())
#     batches = {}; 
#     for i, batch in enumerate(data_in): # Get each batch of trajectories
#         df_batch = pd.read_json(batch, orient = "split")
#         _data = []
#         for _, row in df_batch.iterrows():
#             df_traj = pd.read_json(row["df"], orient = "split")
#             _data.append(df_traj)
#         df_batch["df"] = _data
#         batches[i] = df_batch 
#     return batches

    
# def generate_batches_looping(n_batches):
#     batches = []
#     seeds = [i for i in range(12, 12 + n_batches + 1)]
#     for seed in seeds:
#         waypoints = generate_looping_patterns(
#             seed = seed,
#             n_normal = 10,
#             n_looping = 5, 
#         ) 
#         kwargs = {
#             "mean_time_delta": mean_time_delta,
#             "std_time_delta": std_time_delta,
#             "mean_speed": mean_speed,
#             "std_speed": std_speed,
#             "std_datapoint": 14,
#         }
#         tgc = generate_trajectories(
#             waypoints = waypoints, 
#             start_datetime = datetime.now(),
#             **kwargs,
#         )
#         main_df = tgc.to_dataframe()
#         batches.append(main_df.to_json(orient = "split"))
#     return batches

# batches0 = generate_batches_deviating(n_batches = 1)


def get_first_anom_points(trajectory_generators, pixels = 7.5, consecutive_threshold = 3):
    # ARG 'buffer' should be adjusted the amount of noise in the generated
    # trajectories.
    #
    # Algorithm for finding the point at which a trajectory becomes anomalous:
    # - Get all trajectories termed normal between an origin and a destination
    # - Create a polygon out of these
    # - Find intersection of created polygon and anomalous trajectory
    # --> The first intersection point along the trajectory is when it becomes
    #     anomalous
    # Create separate lists for normal and amomalous trajectories
    norm_traj = [t for v, t in trajectory_generators if v == False]
    anom_traj = [t for v, t in trajectory_generators if v == True]
    # Create 'normal' polygon
    _polygons = [LineString(t.trajectory).buffer(pixels) for t in norm_traj]
#     _polygons = [LineString(t.polylines).buffer(pixels) for t in norm_traj]
    anom_points = {}
    for traj in anom_traj:
        polygon = ops.unary_union(
            _polygons + \
            [LineString(t.trajectory).buffer(pixels) for t in anom_traj if traj.uid != t.uid]
        )
#         print(len(list(traj.trajectory.coords)), len(list(traj.polylines)))
        inter = polygon.intersection(LineString(traj.trajectory)).buffer(pixels)
        diff = LineString(traj.trajectory).difference(inter)
#         return [polygon, inter, diff, LineString(traj.trajectory.coords), traj.uid]
        # anom_point = None
        # Run along trajectory and find when the first
        # anomalous point occurs
        consecutive = []; counter = 0
        for coords in traj.trajectory.coords:
            if diff.contains(Point(coords)):
                consecutive.append(coords)
                # If 'consecutive_threshold' consecutive points are outside the 
                # 'normal' polygon then it must be deviating and the start of an
                # anomalous trajectory
                if len(consecutive) >= consecutive_threshold:
                    # anom_point = consecutive[0]
                    # Adjust counter such that we get corresponding index
                    # of the coordinate in the list of coordinates of the
                    # trajectory
                    counter -= consecutive_threshold - 1
                    break
            else:
                consecutive = []
            counter += 1
        if len(list(traj.trajectory.coords)) <= counter:
            print(len(list(traj.trajectory.coords)), counter)
            return [polygon, inter, diff, LineString(traj.trajectory.coords), traj.uid]
#         print(len(list(traj.trajectory.coords)), counter)
        coord = list(traj.trajectory.coords)[counter]
        lat, lon = utm.to_latlon(coord[0], coord[1], *traj._utm_grid_zone)
        anom_points[traj.uid] = [lat, lon]
    # Return lat/lon coordinates at which anomalous trajectories actually
    # become anomalous 
    return anom_points

vals = get_first_anom_points(trajectories1)

In [12]:
vals

{'75aa583d-68cc-4495-a115-025695a354b6': [55.36716443548277,
  10.397215712372082],
 '10afe2d4-c1ad-41f0-8548-67849fa103e4': [55.368644849793974,
  10.398474674806492],
 '5efe1a5d-5bc6-43b2-b793-543a891d045d': [55.36756364981297,
  10.401132038110564],
 '4ba6e255-a957-4424-b92d-6dd1fd583ac5': [55.36677711100288,
  10.389175103856825],
 '801f17f5-d53a-4a60-a766-ea149d7ad9a4': [55.36645076168086,
  10.389720249474442],
 '1bf3a2b5-a6b4-4f9c-aa43-194998a52486': [55.362787817265726,
  10.404920331314974],
 'c4df442c-8056-4d25-9be6-e52b234124ed': [55.36493449317689,
  10.406143006467879],
 '55e7c50d-7d63-4272-90c9-c564ed8fa8b0': [55.36366529154308,
  10.403235186045409],
 '52ea60b7-9e47-43c1-9563-b03168e35855': [55.36368240731399,
  10.394229950761755],
 'e750462d-cb1b-470d-b4a3-37930f6eb41f': [55.36395537158544,
  10.392763283793998],
 '81b6f8c7-2e0c-4930-8e63-73d34647a717': [55.36698129224795,
  10.423965480345986],
 '73217950-fff1-4a60-8d27-4d8dc7f2b9cb': [55.37494886458261,
  10.41891814

In [13]:
map_ = setup_map(center)

# for item in vals:
#     t = [u for v, u in  trajectories1 if item == u.uid]
#     random.seed(time.time())
#     color = "#" + str(color_hex(num = rand_24_bit()))
#     map_ = plot_trajectory(
#         t[0].to_latlon(),
#         color = color,
#         map_ = map_,
#     )
#     folium.CircleMarker(
#         [vals[item][0], vals[item][1]],
#         radius = 5,
#         color = color,
#         opacity = 1,
#     ).add_to(map_)
# map_

for u, v in trajectories1:
    random.seed(time.time())
    color = "#" + str(color_hex(num = rand_24_bit()))
    map_ = plot_trajectory(
        v.to_latlon(),
        color = color,
        map_ = map_,
    )
    if v.uid in vals:
        folium.CircleMarker(
            [vals[v.uid][0], vals[v.uid][1]],
            radius = 5,
            color = color,
            opacity = 1,
        ).add_to(map_)
map_

In [ ]:
# MultiPolygon([vals[0], vals[1], vals[2].buffer(10)])
# MultiPolygon([vals[0], vals[2]])#, vals[2]])
# vals[2].buffer(10)
# vals[1].buffer(100)
# MultiPolygon([vals[1].buffer(10), vals[2].buffer(10)])
# MultiPolygon([vals[2][1].buffer(25), vals[0]])


# MultiPolygon([vals[2].buffer(25), vals[0]])
# MultiPolygon([vals[-2].buffer(1), vals[2].buffer(25)])
# MultiPolygon([vals[0].buffer(1), vals[2].buffer(10), vals[] ])
# MultiPolygon([vals[0].buffer(1), vals[2].buffer(10), vals[3].buffer(10) ])
MultiPolygon([vals[0].buffer(1)])#, vals[2].buffer(25)]) #, vals[3].buffer(1) ])

# diff = vals[2].buffer(25)
# coords = vals[3]
polygon = vals[0].buffer(5)
inter = vals[1].buffer(5) 
diff = vals[2] #.buffer(5)

MultiPolygon([diff.buffer(10)])#, polygon])#, coords.buffer(0.25)])

# return [polygon, inter, diff, LineString(traj.trajectory.coords), traj.uid]

# vals[2].buffer(25)])
# diff = vals[2].buffer(25)
# traj = vals[-2].coords
# counter = 0; consecutive_threshold = 3
# for coords in traj:
#     if diff.contains(Point(coords)):
#         consecutive.append(coords)
#         # If 'consecutive_threshold' consecutive points are outside the 
#         # 'normal' polygon then it must be deviating and the start of an
#         # anomalous trajectory
#         if len(consecutive) >= consecutive_threshold:
#             # anom_point = consecutive[0]
#             # Adjust counter such that we get corresponding index
#             # of the coordinate in the list of coordinates of the
#             # trajectory
#             counter -= consecutive_threshold - 1
#             break
#     else:
#         consecutive = []
#     counter += 1
# print("Counter", counter)

In [ ]:
# start_seed = 13
# seeds = [2, 3, 5, 6, 8] # [i for i in range(start_seed, start_seed + n_batches + 1)]
# n_batches = len(seeds)
# batches0 = generate_batches_deviating(n_batches = n_batches, seeds = seeds)
# with open("main_data_deviating_large.json", "w") as f:
#     f.write(json.dumps(batches0))

In [11]:
tgc = TrajectoryGeneratorCollection(
    trajectory_generators = trajectories1,
    start_datetime = datetime.now(),
    gap = timedelta(days = 1.),
)
main_df = tgc.to_dataframe(anom_start = True)

In [15]:
batches0 = [main_df.to_json(orient = "split")]
with open("main_data_deviating_large.json", "w") as f:
    f.write(json.dumps(batches0))

In [ ]:
# tgc = TrajectoryGeneratorCollection(
#     trajectory_generators = trajectories1,
#     start_datetime = datetime.now(),
#     gap = timedelta(days = 1.),
# )
# main_df = tgc.to_dataframe(anom_start = True)
# batches0 = [main_df.to_json(orient = "split")]
# with open("segement_test.json", "w") as f:
#     f.write(json.dumps(batches0))